In [1]:

import time
import warnings
import xarray as xr
import numpy as np
from operator import attrgetter
from datetime import timedelta
# from main import LAT_DEG, paths, years_to_days, ofam_fields
# from main import DeleteParticle, plot3D, particle_vars, remove_particles
from parcels import ParticleSet, JITParticle, ScipyParticle, FieldSet
from parcels import AdvectionRK4_3D, Variable, ErrorCode

In [4]:
def DeleteParticle(particle, fieldset, time):
    particle.delete()
def paths():
    unsw = 0 # True if at unsw PC.
    from os.path import expanduser
    home = expanduser("~")
    # Windows Paths.
    if home[:10] == 'C:\\Users\\A':
        path = 'C:/Users/Annette/' if unsw else 'E:/'
        spath = path + 'GitHub/OFAM/scripts/'
        fpath = path + 'GitHub/OFAM/fields/'
        xpath = path + 'GitHub/OFAM/figures/'
        dpath = path + 'GitHub/OFAM/data/'
        data_path = path + 'model_output/OFAM/OFAM3_BGC_SPINUP_03/daily/'

    # Raijin Paths.
    else:
        path = '/g/data/e14/as3189/'
        spath = '/g/data/e14/as3189/OFAM/scripts/'
        fpath = '/g/data/e14/as3189/OFAM/fields/'
        xpath = '/g/data/e14/as3189/OFAM/figures/'
        dpath = '/g/data/e14/as3189/OFAM/data/'
        data_path = '/g/data/e14/as3189/OFAM/'

    return path, spath, fpath, xpath, dpath, data_path
path, spath, fpath, xpath, dpath, data_path = paths()
def ofam_fields(years=[1984, 2014], months=[1, 12], deferred_load=False):

    ufiles = []
    vfiles = []
    wfiles = []
    
    for y in range(years[0], years[1] + 1):
        for m in range(months[0], months[1] + 1):
            ufiles.append('{}trop_pac/ocean_u_{}_{}.nc'.format(data_path, str(y), 
                          str(m).zfill(2)))
            vfiles.append('{}trop_pac/ocean_v_{}_{}.nc'.format(data_path, str(y), 
                          str(m).zfill(2)))
            wfiles.append('{}hist/ocean_w_{}_{}.nc'.format(data_path, str(y), 
                          str(m).zfill(2)))
            
    filenames = {'U': {'lon': ufiles[0], 'lat': ufiles[0], 'depth': ufiles[0], 'data': ufiles},
                 'V': {'lon': ufiles[0], 'lat': ufiles[0], 'depth': ufiles[0], 'data': vfiles},
                 'W': {'lon': ufiles[0], 'lat': ufiles[0], 'depth': ufiles[0], 'data': wfiles}}
    variables = {'U': 'u', 'V': 'v', 'W': 'w'}
    
    dimensions = {'U': {'lon': 'xu_ocean', 'lat': 'yu_ocean', 'depth': 'st_ocean', 'time': 'Time'},
                  'V': {'lon': 'xu_ocean', 'lat': 'yu_ocean', 'depth': 'st_ocean', 'time': 'Time'},
                  'W': {'lon': 'xu_ocean', 'lat': 'yu_ocean', 'depth': 'st_ocean', 'time': 'Time'}}            

    f = FieldSet.from_b_grid_dataset(filenames, variables, dimensions, mesh='flat', 
                                     time_periodic=True, deferred_load=deferred_load)

    return f

In [5]:
years = [2014, 2014]
months = [12, 12]


start = time.time()

# Create the fieldset.
fieldset = ofam_fields(years=years, months=months)
print('Execution time (field): {:.2f} mins'.format((time.time() - start)/60))

Execution time (field): 0.61 mins


In [6]:
ufiles = []
vfiles = []
wfiles = []

for y in range(years[0], years[1] + 1):
    for m in range(months[0], months[1] + 1):
        ufiles.append('{}trop_pac/ocean_u_{}_{}.nc'.format(data_path, str(y), 
                      str(m).zfill(2)))
        vfiles.append('{}trop_pac/ocean_v_{}_{}.nc'.format(data_path, str(y), 
                      str(m).zfill(2)))
        wfiles.append('{}hist/ocean_w_{}_{}.nc'.format(data_path, str(y), 
                      str(m).zfill(2)))
du = xr.open_dataset(ufiles[0])
dv = xr.open_dataset(vfiles[0])
dw = xr.open_dataset(wfiles[0])

In [12]:
print(fieldset.U.eval(0, 7.5, 0.1, 180))
print(np.round(fieldset.W.eval(0, 7.5, 0.1, 222), 9))

-0.23621326736340315
6.104e-06


In [11]:
print(du.u.isel(Time=0).sel(st_ocean=7.5, yu_ocean=0.1, xu_ocean=180, method='nearest'))
print(dw.w.isel(Time=0).sel(sw_ocean=5, yt_ocean=0.1, xt_ocean=222, method='nearest'))

<xarray.DataArray 'u' ()>
array(-0.236213, dtype=float32)
Coordinates:
    Time      object 2014-12-01 12:00:00
    st_ocean  float64 7.5
    xu_ocean  float64 180.0
    yu_ocean  float64 0.1
Attributes:
    long_name:      i-current
    units:          m/sec
    valid_range:    [-32767  32767]
    packing:        4
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_c
    standard_name:  sea_water_x_velocity
<xarray.DataArray 'w' ()>
array(6.103702e-06, dtype=float32)
Coordinates:
    Time      object 2014-12-01 12:00:00
    sw_ocean  float64 5.0
    xt_ocean  float64 222.1
    yt_ocean  float64 0.05
Attributes:
    long_name:      dia-surface velocity T-points
    units:          m/sec
    valid_range:    [-32767  32767]
    packing:        4
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t


In [ ]:
print(dw.sw_ocean)
print(du.st_ocean)

In [ ]:


pset = ParticleSet.from_line(fieldset=fieldset, pclass=JITParticle,
                             size=5,            # releasing 5 particles
                             depth=5,
                             start=(180, -5),   # releasing on a line: the start longitude and latitude
                             finish=(180, 5))  # releasing on a line: the end longitude and latitude



In [ ]:
pset.execute(AdvectionRK4_3D,
             runtime=timedelta(days=10),
             dt=timedelta(minutes=5),
             output_file=pset.ParticleFile(name="GlobCurrentPart.nc", outputdt=timedelta(hours=6)),
            recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

In [ ]:

pset.show(field=fieldset.W)


##### print(du.xu_ocean)
print(dw.xt_ocean)


In [ ]:
print(du.yu_ocean)
print(dw.yt_ocean)

In [ ]:
print(dv.xu_ocean)
print(dw.xt_ocean)

In [ ]:
print(dv.yu_ocean)
print(dw.yt_ocean)


In [ ]:
ufiles = []
vfiles = []
wfiles = []

for y in range(years[0], years[1] + 1):
    for m in range(months[0], months[1] + 1):
        ufiles.append('{}ocean_u_{}_{}_.nc'.format(data_path, str(y), 
                      str(m).zfill(2)))
        vfiles.append('{}ocean_v_{}_{}_.nc'.format(data_path, str(y), 
                      str(m).zfill(2)))
        wfiles.append('{}ocean_w_{}_{}.nc'.format(data_path, str(y), 
                      str(m).zfill(2)))

filenames = {'U': {'lon': ufiles[0], 'lat': ufiles[0], 'depth': ufiles[0], 'data': ufiles},
             'V': {'lon': vfiles[0], 'lat': vfiles[0], 'depth': vfiles[0], 'data': vfiles},
             'W': {'lon': wfiles[0], 'lat': wfiles[0], 'depth': wfiles[0], 'data': wfiles}}
variables = {'U': 'u', 'V': 'v', 'W': 'w'}

dimensions = {'U': {'lon': 'xu_ocean', 'lat': 'yu_ocean', 'depth': 'st_ocean', 'time': 'Time'},
              'V': {'lon': 'xu_ocean', 'lat': 'yu_ocean', 'depth': 'st_ocean', 'time': 'Time'},
              'W': {'lon': 'xt_ocean', 'lat': 'yt_ocean', 'depth': 'sw_ocean', 'time': 'Time'}} 

In [ ]:
du.attrs
if 'U' in dimensions and 'W' in dimensions and dimensions['U'] != dimensions['W']:

In [ ]:
dimensions['W']

In [ ]:
'W' in dimensions

In [13]:
du.attrs

OrderedDict([('filename', 'TMP/ocean_ofam_2014_12.nc.0000'),
             ('NumFilesInSet', 1350),
             ('title', 'jra_55_1979'),
             ('grid_type', 'regular'),
             ('history',
              'Wed Jul 17 17:41:59 2019: ncks -O -d yu_ocean,-15.,14.9 -d xu_ocean,120.,294.9 ocean_u_2014_12.nc ../trop_pac/ocean_u_2014_12.nc\nSun Jul  7 21:06:47 2019: ncks -O -d yu_ocean,-15.0,15.0 -d xu_ocean,120.0,295.0 ocean_u_2014_12.nc trop_pac/ocean_u_2014_12.nc'),
             ('NCO',
              'netCDF Operators version 4.7.7 (Homepage = http://nco.sf.net, Code = http://github.com/nco/nco)')])